In [ ]:
!pip install statsmodels

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import glob
import os
import fnmatch
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib as mplt
import scipy.stats as st
import statsmodels.stats.api as sms
import colorsys
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
import re
import pathlib

output_folder = 'fig'


# MINIIMAGENET, scaling crossvalidation 
# This is with polynomial power learning
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180420_191842_mini_imagenet_metric_multiplier_trainable_encoder_classifier_link_feat_extract_pretrain_polynomial_metric_order_metric_multiplier_init_cbn_per_block_cbn_num_layers_repeat_more_cbn_layers'
# experiment_patterns = [
# #     '*cbn_num_layers=3*polynomial_metric_order=3;feat_extract_pretrain=multitask;encoder_classifier_link=cbn;cbn_per_block=False*',
# #     '*cbn_num_layers=4*polynomial_metric_order=3;feat_extract_pretrain=multitask;encoder_classifier_link=cbn;cbn_per_block=False*',
#     '*cbn_num_layers=5*polynomial_metric_order=3;feat_extract_pretrain=multitask;encoder_classifier_link=cbn;cbn_per_block=False*',
# ]
# This is laets and greatest
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180508_195508_mini_imagenet_num_max_pools_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_init_cbn_num_layers_repeat_scale_maxpool3'
experiment_patterns = [
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*multiplier_init=7.5*',
]
# This is one beofre latest and greatest
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180422_013228_mini_imagenet_metric_multiplier_trainable_encoder_classifier_link_feat_extract_pretrain_polynomial_metric_order_metric_multiplier_init_cbn_per_block_cbn_num_layers_repeat_scaling_crossvalidation_min'
# experiment_patterns = [
#     '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*multiplier_init=5*',
# ]


# # CIFAR100
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180507_015250_mini_imagenet_num_tasks_per_batch_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_init_init_learning_rate_number_of_steps_cbn_num_layers_repeat_scale_crossval_CIFAR100_10shot'
# experiment_patterns = [
#     '*feat_extract_pretrain=multitask;encoder_classifier_link=cbn;number_of_steps=10000*metric_multiplier_init=5',
# ]


fig_name = "beta_gamma_MIN.pdf"

In [ ]:
import tensorflow as tf; print(tf.__version__)

# Get directories for the  experiements 

In [ ]:
def get_experiment_folder_list(experiement_dir, experiment_pattern):
    list_experiemnts_pattern = []
    list_experiments_all = os.listdir(experiement_dir)
    for folder in list_experiments_all:
        names = os.path.join(ROOT_DIR,folder)
#         print(names)
        names_filt_alone = fnmatch.filter([names], experiment_pattern)
#         print(names_filt_alone)
        list_experiemnts_pattern.extend(names_filt_alone)
    return list_experiemnts_pattern


def interpolate_data(experiment_data, index_max, step):
    index_max = int(round(index_max/step)*step)
    index_new = np.arange(0, index_max+step, step)
    experiment_data_interpolated = {}
    for file, df in experiment_data.items():
        interpolator = interpolate.interp1d(df.index, df.value, bounds_error=False, fill_value=np.nan)
        value_new = interpolator(index_new)
        experiment_data_interpolated[file] = value_new
    
    return pd.DataFrame(index=index_new, data=experiment_data_interpolated)
    

def read_experiment_data(experiement_dir, experiment_pattern, variable, step):
    experiemnt_folder_list = get_experiment_folder_list(experiement_dir, experiment_pattern)
    experiment_data_tst = {}
    experiment_data_val = {}
    index_max_tst = 0
    index_max_val = 0
    
    r_gamma = re.compile(".*"+variable+".*")
    
    i=0    
    for experiement in experiemnt_folder_list:
        print(experiement)
        print(glob.glob1(experiement+'/train/', 'events.out.tfevents.*'))
        df_gamma=[]
        df_beta=[]
        for file in glob.glob1(experiement+'/train/', 'events.out.tfevents.*'):
            ea = EventAccumulator(experiement+'/train/'+file)
            ea.Reload()
            tags = ea.Tags()['scalars']
#             print(list(filter(r_gamma.match, tags)))
            if len(list(filter(r_gamma.match, tags))) == 0:
                continue
            
            gamma_list = list(filter(r_gamma.match, tags))
            data = ea.Scalars(gamma_list[0])
            if len(data) > 0:
                df_gamma.append(pd.DataFrame(data).set_index('step').drop('wall_time', axis=1))
        
        if len(df_gamma) > 0:
            df_gamma = pd.concat(df_gamma)
        
        if len(df_gamma) > 0:
            experiment_data_tst[experiement+file] = df_gamma
            index_max_tst = max(index_max_tst, df_gamma.index.max())
        
    return interpolate_data(experiment_data_tst, index_max_tst, step=step)


In [ ]:
list_experiemnts_target_transfer = get_experiment_folder_list(ROOT_DIR, experiment_patterns[0])
list_experiemnts_target_transfer

In [ ]:
plot_data = {}
for experiment_pattern in experiment_patterns:
    print('Loading experiment: %s' %experiment_pattern)
    
    gamma_weight = read_experiment_data(ROOT_DIR, experiment_pattern, 
                                        variable='gamma_weight00', step=500)
    
    gamma_weight_layers = pd.DataFrame(index=np.arange(12), columns=gamma_weight.columns)
    beta_weight_layers = pd.DataFrame(index=np.arange(12), columns=gamma_weight.columns)
    for i in range(4):
        for j in range(3):
            gamma_weight = read_experiment_data(ROOT_DIR, experiment_pattern, 
                                                variable='gamma_weight'+str(i)+str(j), step=500)
            gamma_weight_layers.iloc[3*i+j]=gamma_weight.iloc[-2]
            
            beta_weight = read_experiment_data(ROOT_DIR, experiment_pattern, 
                                               variable='beta_weight'+str(i)+str(j), step=500)
            beta_weight_layers.iloc[3*i+j]=beta_weight.iloc[-2]
    
    plot_data[(experiment_pattern, 'gamma_weight')] = gamma_weight_layers
    plot_data[(experiment_pattern, 'beta_weight')] = beta_weight_layers


In [ ]:
pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)

mplt.rcParams.update({'font.size': 22})

fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)
plt.grid('on')

ax1.boxplot(abs(plot_data[(experiment_patterns[0], 'gamma_weight')].T.values))
ax1.grid(which='major', alpha=0.5) 
# ax1.set_xlabel('Conv layer #')
ax1.set_ylabel(r'$\gamma_0$')

ax2.boxplot(abs(plot_data[(experiment_patterns[0], 'beta_weight')].T.values))
ax2.grid(which='major', alpha=0.5) 
ax2.set_ylabel(r'$\beta_0$')
ax2.set_xlabel('Conv layer #')

fig.savefig(os.path.join(output_folder,fig_name), bbox_inches='tight')

In [ ]:
def draw_plot(data, edge_color, fill_color, ax):
    bp = ax.boxplot(data, patch_artist=True)

    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color) 
    return bp

In [ ]:
mplt.rcParams.update({'font.size': 22})

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)
ax.set_yscale('log')
plt.grid('on')

bp_gamma = draw_plot(abs(plot_data[(experiment_patterns[0], 'gamma_weight')].T.values), 'red', 'tan', ax)
bp_beta = draw_plot(abs(plot_data[(experiment_patterns[0], 'beta_weight')].T.values), 'blue', 'cyan', ax)
ax.set_xlabel('Conv layer #')
ax.set_ylim([3e-6, 0.6])

ax.legend([bp_gamma["boxes"][0], bp_beta["boxes"][0]], [r'$\gamma_0$', r'$\beta_0$'], loc='upper left')

fig.savefig(os.path.join(output_folder,fig_name), bbox_inches='tight')